<a href="https://colab.research.google.com/github/xmfaizan/nlp-lstm/blob/main/bi_lstm_junknews_classif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# same dataset using Bidirectional LSTM RNN (accuracy increased by 2%)

In [1]:
import pandas as pd

In [23]:
df = pd.read_csv('train.csv')

In [27]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [28]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [30]:
df.shape

(20800, 5)

In [38]:
df = df.dropna()

In [39]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [40]:
X = df.drop('label',axis=1)

In [41]:
y = df['label']

In [42]:
y.value_counts

<bound method IndexOpsMixin.value_counts of 0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64>

In [11]:
import tensorflow as tf

In [12]:
tf.__version__

'2.15.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [15]:
vocab_size = 10000

In [43]:
msgs = X.copy()

In [45]:
msgs.reset_index(inplace=True)

In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [48]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(msgs)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', msgs['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [49]:
review, corpus

('keep f aliv',
 ['hous dem aid even see comey letter jason chaffetz tweet',
  'flynn hillari clinton big woman campu breitbart',
  'truth might get fire',
  'civilian kill singl us airstrik identifi',
  'iranian woman jail fiction unpublish stori woman stone death adulteri',
  'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
  'beno hamon win french socialist parti presidenti nomin new york time',
  'back channel plan ukrain russia courtesi trump associ new york time',
  'obama organ action partner soro link indivis disrupt trump agenda',
  'bbc comedi sketch real housew isi caus outrag',
  'russian research discov secret nazi militari base treasur hunter arctic photo',
  'us offici see link trump russia',
  'ye paid govern troll social media blog forum websit',
  'major leagu soccer argentin find home success new york time',
  'well fargo chief abruptli step new york time',
  'anonym donor pay million releas everyon arrest dakota ac

In [51]:
ohe=[one_hot(words,vocab_size) for words in corpus]
ohe

[[354, 8753, 4581, 3636, 8655, 9050, 6810, 1110, 5416, 8610],
 [2605, 6101, 7059, 8093, 8819, 7305, 3872],
 [2590, 2254, 4328, 8859],
 [5949, 9189, 691, 8553, 115, 4016],
 [2319, 8819, 4211, 2082, 2998, 6699, 8819, 8760, 9436, 4530],
 [3074,
  6095,
  3903,
  6863,
  4420,
  9863,
  6901,
  5225,
  8141,
  2703,
  633,
  9968,
  6820,
  4592,
  3872],
 [9536, 6224, 4734, 3169, 4398, 7989, 1265, 4413, 9236, 4030, 7781],
 [5154, 3562, 6208, 5112, 2150, 4905, 9863, 2806, 9236, 4030, 7781],
 [6236, 585, 5055, 9816, 6413, 146, 1720, 388, 9863, 1952],
 [5987, 6788, 2339, 1590, 7320, 1436, 6113, 989],
 [1261, 472, 8171, 8028, 5707, 9901, 8132, 7680, 5376, 4489, 3153],
 [8553, 939, 8655, 146, 9863, 2150],
 [7358, 6433, 3348, 2900, 5611, 5887, 4304, 7375, 1535],
 [427, 6339, 4320, 476, 9181, 6644, 4874, 9236, 4030, 7781],
 [8985, 5096, 875, 5129, 2369, 9236, 4030, 7781],
 [9817, 3403, 3237, 8621, 3446, 6048, 9206, 4806, 8473, 9655],
 [8004, 4854, 6101],
 [9184, 3644, 771, 6335, 9863, 1002, 5957

In [52]:
maxsent_l=15
embedded_docs=pad_sequences(ohe,padding='pre',maxlen=maxsent_l)

embedded_docs

array([[   0,    0,    0, ..., 1110, 5416, 8610],
       [   0,    0,    0, ..., 8819, 7305, 3872],
       [   0,    0,    0, ..., 2254, 4328, 8859],
       ...,
       [   0,    0,    0, ..., 9236, 4030, 7781],
       [   0,    0,    0, ..., 8553, 8916, 1450],
       [   0,    0,    0, ..., 3661, 5166,  417]], dtype=int32)

In [53]:
## Creating model
embedding_vector_features=30
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=maxsent_l))
model.add(Bidirectional(LSTM(100))) #100 neurons forward and backward
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 30)            300000    
                                                                 
 bidirectional (Bidirection  (None, 200)               104800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 405001 (1.54 MB)
Trainable params: 405001 (1.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [54]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [55]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=20)

In [56]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=64)

Epoch 1/8
215/215 [==============================] - 21s 70ms/step - loss: 0.2779 - accuracy: 0.8676 - val_loss: 0.1943 - val_accuracy: 0.9206
Epoch 2/8
215/215 [==============================] - 3s 16ms/step - loss: 0.1214 - accuracy: 0.9535 - val_loss: 0.1966 - val_accuracy: 0.9232
Epoch 3/8
215/215 [==============================] - 3s 13ms/step - loss: 0.0733 - accuracy: 0.9729 - val_loss: 0.2280 - val_accuracy: 0.9208
Epoch 4/8
215/215 [==============================] - 4s 19ms/step - loss: 0.0465 - accuracy: 0.9841 - val_loss: 0.2750 - val_accuracy: 0.9250
Epoch 5/8
215/215 [==============================] - 3s 14ms/step - loss: 0.0263 - accuracy: 0.9913 - val_loss: 0.3567 - val_accuracy: 0.9178
Epoch 6/8
215/215 [==============================] - 3s 12ms/step - loss: 0.0178 - accuracy: 0.9940 - val_loss: 0.3436 - val_accuracy: 0.9173
Epoch 7/8
215/215 [==============================] - 2s 11ms/step - loss: 0.0077 - accuracy: 0.9979 - val_loss: 0.5386 - val_accuracy: 0.9202
Epoch

In [57]:
y_pred=model.predict(X_test)

143/143 [==============================] - 1s 3ms/step


In [58]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [59]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2306,  263],
       [ 102, 1901]])

In [60]:
print(accuracy_score(y_test,y_pred))

0.9201662292213474


In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      2569
           1       0.88      0.95      0.91      2003

    accuracy                           0.92      4572
   macro avg       0.92      0.92      0.92      4572
weighted avg       0.92      0.92      0.92      4572

